In [1]:
import numpy as np
import numba as nb
import matplotlib.pyplot as plt
from numba import njit, prange
import random
###########################################################################################################################
@njit(fastmath=True, parallel=True)
def estimate_pi_vectorized(N):
    x = np.random.rand(N)
    y = np.random.rand(N)
    distances = x**2 + y**2
    num_inside_circle = np.sum(distances <= 1.0)
    res = 4.0 * num_inside_circle / N
    return res, res-np.std(4.0*(distances <= 1.0))/np.sqrt(N), res+np.std(4.0*(distances <= 1.0))/np.sqrt(N)
###########################################################################################################################
@njit(fastmath=True, parallel=True)        
def estPiNaiveMC(N):
    tot=0
    for _ in prange(N):
        U = np.random.uniform()
        tot += np.sqrt(1 - U**2)   
    return 4*tot/N
###########################################################################################################################
@njit(fastmath=True, parallel=True, nogil=True)        
def estPiNaiveMC2(N): 
    arr = np.array([np.sqrt(1 - np.random.rand()**2) for _ in prange(N)])
    res = 4.0 * np.mean(arr)
    return res, res-np.std(4.0*arr)/np.sqrt(N), res+np.std(4.0*arr)/np.sqrt(N)
###########################################################################################################################
@njit(fastmath=True, parallel=True)
def estPiStratified(N):
    U = np.random.rand(N)
    jVet = np.arange(N)
    X = np.sqrt(1 - ((U + jVet) / N)**2)
    res =  4 * np.mean(X)
    return res, res-np.std(4 * X)/np.sqrt(N), res+np.std(4 * X)/np.sqrt(N)
###########################################################################################################################
@njit(fastmath=True, parallel=True)
def estPiStratifiedAnti(N):
    U = np.random.rand(N)
    jVet = np.arange(1, N+1)
    X = np.sqrt(1-((U+jVet-1)/N)**2) + np.sqrt(1-((jVet-U)/N)**2)
    res = 2 * np.mean(X)
    return  res, res-np.std(2*X)/np.sqrt(N), res+np.std(2*X)/np.sqrt(N)
###########################################################################################################################
@njit(fastmath=True, parallel=True)
def estPi_IS(m, L):
    # define left end points of subintervals
    s = np.linspace(0, 1 - 1/L, L) + 1/(2*L)
    hvals = np.sqrt(1 - s**2)
    
    # get cumulative probabilities
    cs = np.cumsum(hvals)
    est = np.zeros(m)
    
    for j in prange(m):
        # locate subinterval
        loc = np.sum(np.random.uniform() * cs[L-1] > cs)
        
        # sample uniformly within subinterval
        x = (loc+np.random.rand())/L
        p = hvals[loc]/cs[L-1]
        
        est[j] = np.sqrt(1 - x**2)/(p*L)
    res = 4*np.mean(est)
    return res, res-np.std(4*est)/np.sqrt(m), res+np.std(4*est)/np.sqrt(m)
###########################################################################################################################@njit(fastmath=True, parallel=True)
@njit(fastmath=True, parallel=True)
def estPi_ISAnti(m, L):
    # define left end points of subintervals
    s = np.linspace(0, 1-1/L, L) + 1/(2*L)
    hvals = np.sqrt(1 - s**2)
    
    # get cumulative probabilities
    cs = np.cumsum(hvals)
    est = np.zeros(m)
    
    for j in prange(m):
        # locate subinterval
        loc = np.sum(np.random.rand() * cs[L-1] > cs)

        # sample uniformly within subinterval
        U = np.random.rand()
        x1 = (loc+U)/L
        x2 = (loc+(1-U))/L
        p = hvals[loc]/cs[L-1]
        
        est[j] = np.sqrt(1 - x1**2)/(p*L) + np.sqrt(1 - x2**2)/(p*L)
    res = 2.0*np.sum(est)/m
    return res, res-np.std(2.0*est)/np.sqrt(m), res+np.std(2.0*est)/np.sqrt(m)
###########################################################################################################################@
@njit(fastmath=True, parallel=True)
def estPiStratifiedAnti2(N):
    U = np.random.rand(N)
    jVet = np.arange(1, N+1)
    V1 = (U+jVet-1)/N
    V2 = (jVet-U)/N
    X1 = 1.0/(1+(V1)**2)
    X2 = 1.0/(1+(V2)**2)
    Y1 = np.sqrt(1-(V1)**2) 
    Y2 = np.sqrt(1-(V2)**2)
    XY = X1+X2+Y1+Y2 
    res = np.mean(XY)
    return  res, res-np.std(XY)/np.sqrt(N), res+np.std(XY)/np.sqrt(N)

In [2]:
e8 = int(1e8)
e7 = int(1e7)
%timeit -r 1 estPiNaiveMC(e8)
print(estPiNaiveMC(e8))

%timeit -r 1 estimate_pi_vectorized(e8)
print(estimate_pi_vectorized(e8))
%timeit -r 1 estPiNaiveMC2(e8)
print(estPiNaiveMC2(e8))

%timeit -r 1 estPiStratified(e7)
print(estPiStratified(e7))

%timeit -r 1 estPiStratifiedAnti(e7)
print(estPiStratifiedAnti(e7))

%timeit -r 1 estPi_IS(1000000, 20000)
print(estPi_IS(1000000, 20000))

%timeit -r 1 estPi_ISAnti(1000000, 20000)
print(estPi_IS(1000000, 20000))

%timeit -r 1 estPiStratifiedAnti2(2*e7)
print(estPiStratifiedAnti2(2*e7))

99.3 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
3.141672619807262
1.05 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
(3.1414316, 3.1412673704684737, 3.1415958295315267)
959 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
(3.1417655431322915, 3.141676274986114, 3.141854811278469)
266 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
(3.141592653562843, 3.1413103306510033, 3.141874976474683)
108 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
(3.1415926535923724, 3.1413103306805215, 3.1418749765042233)
285 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
(3.1415925443201727, 3.141592202830259, 3.1415928858100863)
277 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
(3.141593971737285, 3.141593033462357, 3.141594910012213)
110 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
(3.1415926535911027, 3.1414233875681328, 3.1417619196140727)


In [3]:
np.pi

3.141592653589793